In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import typing

In [ ]:
class AttentionHeadConfig(typing.NamedDict):
    # Dimension of the embedding of each token
    d_embed: int 
    # Dimension of the key, query and value vectors
    d_k: int
    # Size of the input sequence
    ctx_len: int

class AttentionHead(nn.Module):
    def __init__(self, config: AttentionHeadConfig):
        super().__init__()
        self.config = config
        # linear layers to project the input to the key, query and value vectors

        self.q = nn.Linear(config.d_embed, config.d_k, bias=False)
        self.k = nn.Linear(config.d_embed, config.d_k, bias=False)
        self.v = nn.Linear(config.d_embed, config.d_k, bias=False)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(config.ctx_len, config.ctx_len)))
    
    def forward(self, x):
        # x in (batch_size, ctx_len, d_embed)
        # q in (batch_size, ctx_len, d_k)
        q = self.q(x)
        # k in (batch_size, ctx_len, d_k)
        k = self.k(x)

        # masked self attention
        # a in (batch_size, ctx_len, ctx_len)
        a = (q @ k.transpose(-2, -1)) / (self.config.d_k ** 0.5)
        a = a.masked_fill(self.mask == 0, float("-inf"))
        a = F.softmax(a, dim=-1)
        
        # v in (batch_size, ctx_len, d_k)
        v = self.v(x)
        
        # att in (batch_size, ctx_len, d_k)        
        att = a @ v
        return att

In [ ]:
class MultiHeadAttentionConfig(typing.NamedDict):
    # Number of attention heads
    n_heads: int
    # Dimension of the output vector
    d_out: int
    # Dimension of the embedding of each token
    d_embed: int 
    # Dimension of the key, query and value vectors
    d_k: int
    # Size of the input sequence
    ctx_len: int

class MultiHeadAttention(nn.Module):
    def __init__(self, config: MultiHeadAttentionConfig):
        super().__init__()
        self.config = config
        self.heads = nn.ModuleList([
            AttentionHead(AttentionHeadConfig(
                d_embed=config.d_embed,
                d_k=config.d_k,
                ctx_len=config.ctx_len
            )) for _ in range(config.n_heads)
        ])
        self.o = nn.Linear(config.n_heads*config.d_k, config.d_out, bias=False)
    
    def forward(self, x):
        return self.o(torch.cat([head(x) for head in self.heads], dim=-1))

In [ ]:
class TransformerBlockConfig(typing.NamedDict):
    # Dimension of the embedding of each token
    d_embed: int 
    # Dimension of the key, query and value vectors
    d_k: int
    # Size of the input sequence
    ctx_len: int
    # Number of attention heads
    n_heads: int
    # Number of layers in the feed-forward network
    n_ff_layers: int


class TransformerBlock(nn.Module):
    def __init__(self, config: TransformerBlockConfig):
        super().__init__()
        self.config = config
        self.attn = MultiHeadAttention(MultiHeadAttentionConfig(
            d_embed=config.d_embed,
            d_k=config.d_k,
            ctx_len=config.ctx_len,
            n_heads=config.n_heads,
            d_out=config.d_embed
        ))
        self.ln1 = nn.LayerNorm(config.d_embed)
        self.ff = nn.Sequential(
            nn.Linear(config.d_embed, config.d_embed * config.n_ff_layers),
            nn.ReLU(),
            nn.Linear(config.d_embed * config.n_ff_layers, config.d_embed)
        )
        self.ln2 = nn.LayerNorm(config.d_embed)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x):
        # x in (batch_size, ctx_len, d_embed)
        # att in (batch_size, ctx_len, d_embed)
        att = self.attn(x)
        # x in (batch_size, ctx_len, d_embed)
        x = self.ln1(x + self.dropout(att))
        # x in (batch_size, ctx_len, d_embed)
        x = self.ln2(x + self.dropout(self.ff(x)))
        return x